In [1]:
import os

def find_project_root(search_path, target_file="params.yaml"):
    print(f"Searching for {target_file} in {search_path}... Please wait.")
    for root, dirs, files in os.walk(search_path):
        if target_file in files:
            # Check if this folder also has 'config' to be sure it's the project root
            if "config" in dirs or "artifacts" in dirs:
                return root
    return None

# This starts searching from your main user folder
user_path = os.path.expanduser("~")
found_path = find_project_root(user_path)

if found_path:
    print(f"\n🚀 PROJECT FOUND!")
    print(f"COPY THIS PATH: {found_path}")
else:
    print("\n❌ Project folder not found. Are you sure it is downloaded?")

Searching for params.yaml in C:\Users\sheet... Please wait.

🚀 PROJECT FOUND!
COPY THIS PATH: C:\Users\sheet\Downloads\OneDrive\Desktop\ML_Project\Chest-Cancer-Classification-ML-Flow-DVC


In [2]:
# Replace the text below with the path the script just found
os.chdir(r"C:\Users\sheet\Downloads\OneDrive\Desktop\ML_Project\Chest-Cancer-Classification-ML-Flow-DVC")
print(f"Current Directory: {os.getcwd()}")

Current Directory: C:\Users\sheet\Downloads\OneDrive\Desktop\ML_Project\Chest-Cancer-Classification-ML-Flow-DVC


In [3]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/sheetalgat-hubGit/Chest-Cancer-Classification-ML-Flow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "sheetalgat-hubGit"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "cfd855ba71d40aa6fa36de270177e07a787ba1bc"


In [4]:
import dagshub
import mlflow

c:\Users\sheet\miniconda3\envs\chestcancer\lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sheetalgat-hubGit/Chest-Cancer-Classification-ML-Flow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sheetalgat-hubGit"
os.environ["MLFLOW_TRACKING_PASSWORD"]="
"


In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [7]:

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/sheetalgat-hubGit/Chest-Cancer-Classification-ML-Flow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        mlflow.set_experiment("Chest-Cancer-Classification-v1")
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [ ]:
# class Evaluation:
#     def __init__(self, config: EvaluationConfig):
#         self.config = config

    
#     def _valid_generator(self):

#         datagenerator_kwargs = dict(
#             rescale = 1./255,
#             validation_split=0.30
#         )

#         dataflow_kwargs = dict(
#             target_size=self.config.params_image_size[:-1],
#             batch_size=self.config.params_batch_size,
#             interpolation="bilinear"
#         )

#         valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
#             **datagenerator_kwargs
#         )

#         self.valid_generator = valid_datagenerator.flow_from_directory(
#             directory=self.config.training_data,
#             subset="validation",
#             shuffle=False,
#             **dataflow_kwargs
#         )


#     @staticmethod
#     def load_model(path: Path) -> tf.keras.Model:
#         return tf.keras.models.load_model(path)
    

#     def evaluation(self):
#         self.model = self.load_model(self.config.path_of_model)
#         self._valid_generator()
#         self.score = model.evaluate(self.valid_generator)
#         self.save_score()

#     def save_score(self):
#         scores = {"loss": self.score[0], "accuracy": self.score[1]}
#         save_json(path=Path("scores.json"), data=scores)

    
#     def log_into_mlflow(self):
#         dagshub.init(repo_owner='sheetalgat-hubGit', 
#                  repo_name='Chest-Cancer-Classification-ML-Flow-DVC', 
#                  mlflow=True)
#         mlflow.set_experiment("Chest-Cancer-Classification-v1")
        
#         with mlflow.start_run():
#             mlflow.log_params(self.config.all_params)
#             mlflow.log_metrics(
#                 {"loss": self.score[0], "accuracy": self.score[1]}
#             )
#             mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16_Chest_Model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2026-01-28 21:14:58,655: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-28 21:14:58,669: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-28 21:14:58,672: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 8s 1s/step - loss: 0.3939 - accuracy: 0.9706
[2026-01-28 21:15:07,080: INFO: common: json file saved at: scores.json]


2026/01/28 21:15:10 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-01-28 21:15:11,169: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\sheet\AppData\Local\Temp\tmpibp7a5y_\model\data\model\assets
[2026-01-28 21:15:11,720: INFO: builder_impl: Assets written to: C:\Users\sheet\AppData\Local\Temp\tmpibp7a5y_\model\data\model\assets]


2026/01/28 21:15:21 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-02-28; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'backports-tarfile'}
c:\Users\sheet\miniconda3\envs\chestcancer\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2026/01/28 21:16:25 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.
